In [1]:
import numpy as np
import pandas as pd
import transformers
import torch

In [2]:
! pip install datasets transformers

In [3]:
import transformers
print(transformers.__version__)

4.31.0


In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
from transformers.utils import send_example_telemetry

send_example_telemetry("language_modeling_notebook", framework="pytorch")

In [6]:
from datasets import load_dataset
datasets = load_dataset('text', data_files={'train': "../data/train_data.txt", 'validation': "../data/test_data.txt"})

In [7]:
datasets['train'][10]

{'text': "customer: Hi, is this jacket dry clean only? agent: Hello, it is not dry clean only. We use high quality fabric in our materials, so you can wash and dry them in a typical washer and dryer! customer: Great! It is a little inconvenient when one is not machine washable. agent: I highly agree! Especially with everything happening right now, you may not even be able to find one that is open! agent: Anyway, is there anything else I can help you with? customer: Do they come in different colors agent: Hello, I would have to direct you to our website, as our stock is very dynamic. If you do not see the color you like, I suggest checking back at the end of the month, when we refill the stock customer: That's alright. I actually want it in red if you have it. But if not, I'll get this. agent: That sounds great! Is there anything else I can help you with? customer: this is good. thank you agent: Have a great day!"}

In [8]:
from datasets import ClassLabel
import random
from IPython.display import display,HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more"
    picks=[]
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0,len(dataset)-1)
        picks.append(pick)
        
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [9]:
show_random_elements(datasets['train'])

,text
0,"agent: AcmeBrands, how may I help you? customer: I think I may have a subscription that I didn't order agent: Let me help you with that. agent: Could I have your full name or account id? customer: Rodriguez Domingo agent: How did you hear about that subscription? customer: I heard some news about it agent: Let me check the system. agent: There is a subscription on your account. agent: Let me remove it from your account. customer: okay, thanks customer: cool, will I get a refund? agent: That subscription has been removed from your account. agent: How much was the subscription, so that I can refund the amount? customer: It should be $40 customer: great, thanks agent: I apologize for the error. Your refund should be in your account. agent: Can I help you with anything else? customer: cool, thanks, that's all agent: Have a good day."
1,"agent: Hi, how are you doing today? customer: I got an email that seemed to display a different item than I ordered customer: can you confirm that the item being delivered is michael kors boots? agent: Sure, I can look into that for you. agent: What is your name? customer: Chloe Zhang agent: Thank you, can I also have your account ID and order ID agent: Along with those two items, I will also need your username and email address please customer: czhang99 czhang99@email.com agent: Can I also have your account ID and order ID? customer: FBPJ0UKPVS 2970875983 agent: Thank you, one moment while I check the system please agent: I apologize, it looks like the email was correct. I will change the order in our system to the Michael Kors boots for you. customer: thanks agent: You're welcome, is there anything else I can help you with today? customer: no that was all agent: Have a great day!"
2,"customer: hello? agent: Hello, how can I help you? customer: Yes, I want to check on the status of my premium subscription shopping service. Do I need to make a payment before it is activated? agent: Let me check that for you, can I have your full name, account ID, and order ID/ customer: Joseph Banter customer: Account ID: NJ8N9YQDWP customer: Order ID: 0020283275 agent: Your subscription is already active, but it will be deactivated tomorrow if you do not make a payment by tomorrow. customer: ok, how much do i owe? agent: Your fee is $55, I will send you a link that shows this information to you next time. All you need is your username. customer: ok great. thank you agent: You're welcome, is there anything else you need? customer: nope, that covers it. thanks agent: Okay, thank you and have a nice day."
3,agent: Hello. How may I help you today? customer: I'm looking at these Calvin Klein boots. Are the laces Brown or Black? agent: One moment while I look into this for you please. agent: Most of our boots have blacks laces. About 90% of them. agent: You can see on the product page what color they are. agent: Does that answer your question? agent: Is there anything else I can help you with? agent: Have a great day.
4,"agent: How can I help you? customer: Hello, I am trying to browse the site and it is going really slow for me agent: Sorry to hear that. Can you please try refreshing? customer: I tried that and did not work agent: I'm sorry try to log out and back in is what I mean. customer: I did that as well agent: I can file a report with our website team so they can look into it. customer: Ok that would be great agent: Can you please closing out all other tabs, that will sometimes slow things down. customer: That seemed to work for me, thank you for the help agent: You're welcome. Can I help with anything else? customer: That is all"
5,"agent: Hello, thank you for contacting AcmeCorp how may I help you? customer: I am expecting a $84 refund and just wanted to check on the status of that refund agent: I would happy to look into this for you. agent: May I have your full name or Account ID please? customer: David Williams agent: Thank you David, may I please have your username, email address, and Order ID? c

In [10]:
model_checkpoint='gpt2'

In [11]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token':'[PAD]'})

Using pad_token, but it is not set yet.


In [12]:
def tokenize_function(examples,tokenizer=tokenizer):
    return tokenizer(examples['text'])

In [13]:
tokenized_datasets= datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=['text'])


Map (num_proc=4): 100%|██████████| 1004/1004 [00:07<00:00, 133.62 examples/s]


In [14]:
tokenized_datasets['train'][1]

{'input_ids': [25781,
  25,
  922,
  6672,
  11,
  703,
  460,
  314,
  1037,
  345,
  30,
  6491,
  25,
  655,
  2227,
  284,
  2198,
  319,
  262,
  3722,
  286,
  257,
  12929,
  5797,
  25,
  1654,
  11,
  561,
  345,
  1577,
  502,
  534,
  1336,
  1438,
  393,
  1848,
  4522,
  6491,
  25,
  47319,
  28092,
  9643,
  6491,
  25,
  49812,
  8538,
  24,
  2670,
  5797,
  25,
  3224,
  284,
  428,
  345,
  561,
  1577,
  502,
  262,
  1502,
  4522,
  290,
  3053,
  5797,
  25,
  3387,
  6491,
  25,
  9225,
  1433,
  3134,
  2414,
  1983,
  6491,
  25,
  49812,
  8538,
  24,
  2670,
  31,
  12888,
  13,
  785,
  6491,
  25,
  645,
  18572,
  5797,
  25,
  2279,
  287,
  1502,
  11,
  2582,
  314,
  481,
  7603,
  262,
  3722,
  286,
  534,
  12929,
  13,
  6491,
  25,
  1049,
  6491,
  25,
  1309,
  502,
  760,
  5797,
  25,
  632,
  318,
  3058,
  287,
  4371,
  290,
  262,
  6074,
  2446,
  351,
  543,
  340,
  318,
  852,
  13686,
  318,
  2691,
  3371,
  534,
  3884,
  2657,
  13

In [15]:
#block_size=tokenizer.model_max_length #1024
block_size=128


In [16]:

def group_texts(examples, block_size=block_size):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [17]:

lm_datasets = tokenized_datasets.map(group_texts, batched=True, batch_size=1000,  num_proc=4, )


Map (num_proc=4): 100%|██████████| 1004/1004 [00:05<00:00, 178.75 examples/s]


In [18]:
tokenizer.decode(lm_datasets['train'][1]['input_ids'])

" a bronze agent: ok, was the purchase made in the last 90 days? customer: No, I bought it in November. agent: ok, unfortunately because it has been more than 90 days we cannot accept the return. Would there be anything else I can help you with? customer: What if I ask really, really nicely? agent: I can escalate to my manager if you'd like agent: I'd just need your phone number. customer: (977) 625-2661 customer: I'll look forward to hearing from them. customer: Thanks for trying to help. agent: OK, I have let my manager know"

In [19]:
#Training Configuration
# Add AutoModelForCasualLM-

from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

# Tokenizer is not defined
from transformers import Trainer, TrainingArguments


In [20]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    output_dir=f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

In [21]:
# HTTPError
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets['train'],
    eval_dataset=lm_datasets['validation'],
    tokenizer=tokenizer,
    )

Cloning https://huggingface.co/yeseok/gpt2-finetuned-wikitext2 into local empty directory.


In [22]:
# HTTP Error, RepositoryNotFoundError bug
import huggingface_hub
print(huggingface_hub.__version__)

0.16.4


In [23]:
# Model check
model.__class__.__name__

'GPT2LMHeadModel'

In [24]:
# Train data
trainer.train()


c:\Users\dotto\anaconda3\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  9%|▉         | 500/5283 [3:04:06<19:43:47, 14.85s/it]   

{'loss': 2.5468, 'learning_rate': 1.8107136096914633e-05, 'epoch': 0.28}


 19%|█▉        | 1000/5283 [5:34:38<27:45:03, 23.33s/it]

{'loss': 2.2647, 'learning_rate': 1.6214272193829265e-05, 'epoch': 0.57}


 28%|██▊       | 1500/5283 [7:59:56<21:27:28, 20.42s/it]

...

 95%|█████████▍| 5000/5283 [36:52:59<1:41:03, 21.43s/it]

{'loss': 1.9697, 'learning_rate': 1.0713609691463186e-06, 'epoch': 2.84}


                                                        
100%|██████████| 5283/5283 [38:52:44<00:00, 26.49s/it]

{'eval_loss': 1.9649392366409302, 'eval_runtime': 1031.8592, 'eval_samples_per_second': 1.706, 'eval_steps_per_second': 0.213, 'epoch': 3.0}
{'train_runtime': 139964.5478, 'train_samples_per_second': 0.302, 'train_steps_per_second': 0.038, 'train_loss': 2.1058111037355967, 'epoch': 3.0}


TrainOutput(global_step=5283, training_loss=2.1058111037355967, metrics={'train_runtime': 139964.5478, 'train_samples_per_second': 0.302, 'train_steps_per_second': 0.038, 'train_loss': 2.1058111037355967, 'epoch': 3.0})

In [25]:
#Masked language modeling
model_checkpoint = "distilroberta-base"


In [26]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns='text')

Map (num_proc=4): 100%|██████████| 1004/1004 [00:11<00:00, 89.65 examples/s]


In [27]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4): 100%|██████████| 1004/1004 [00:05<00:00, 187.35 examples/s]


In [28]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [29]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

In [30]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [33]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

100%|██████████| 220/220 [13:29<00:00,  3.68s/it]


In [34]:
#Save Fine-tuned 
trainer.save_model('../data/fine_tuned_1201')

Upload file pytorch_model.bin: 488MB [05:42, 2.03MB/s]                            To https://huggingface.co/yeseok/gpt2-finetuned-wikitext2
   29c6780..eb2f544  main -> main

Upload file pytorch_model.bin: 100%|██████████| 475M/475M [05:43<00:00, 1.45MB/s]
To https://huggingface.co/yeseok/gpt2-finetuned-wikitext2
   eb2f544..df116bc  main -> main



In [35]:
trainer.push_to_hub()